# 네이버 기사 크롤링 테스트
## 네이버 기사 사회 - 교육면에서 모든 날짜의 기사를 크롤링
### 기본 라이브러리 임포트

In [10]:
import urllib.request
from bs4 import BeautifulSoup

import os #환경 변수나 디렉터리, 파일 등의 OS 자원을 제어할 수 있게 해주는 모듈
import sys #파이썬 파일을 실행할 때, 자기 자신의 파일명이 들어간다
import datetime #날짜관련 모듈
import csv

네이버 사회면 : sid1 = 102,<br>
사회면 중 교육: sid2 = 250

In [11]:
url = 'https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=102&sid2=250&listType=title'

크롤링한 데이터를 저장할 파일 설정 - csv형태로 저장<br>
open(파일경로, 모드)<br>
모드 : r - 파일읽기 / w - 파일에 기록 / a - 파일에 데이터 추가 / x - 파일에 기록

In [12]:
filename = 'NAVER1.csv'
f = open(filename, 'w', encoding = 'utf-8', newline = '')
wr = csv.writer(f)
print(filename)

NAVER1.csv


### 각 월별로 크롤링할때 월마다 날짜가 다르기 때문에 설정해야됨
#### 네이버 기사에서 날짜를 1월1일일때 0101로 인식하기 때문에 
#### 10이하 월/일 앞에 0을 붙여줘야함

* <월별로 최대 날짜 설정>
    * 1,3,5,7,8,10,12 월은 31일
    * 4,6,9,11 월은 30일
    * 2월은 28일

***
## 첫 코드 원본

```
global days
global url_date

#def zeroday():
#    if days<10:             #날짜가 10 이하일경우 앞에 0 추가
#        Bdate = '0'+str(days)
#    else:                   #그외 날짜는 그대로 저장
#        Bdate = str(days)
        
def urlstack():
    date = '2019' + str(Adate) + str(Bdate)
    url_date = url + '&date=' + date
    print(url_date)

for mons in range(1,13,1):      #최대 12얼까지 있기때문에 13으로 설정
    if mons<10:                 #월이 10 이하일 경우 앞에 0 추가
        Adate = '0' + str(mons)
    else:
        Adate = str(mons)
    
    if mons%2 != 0 and mons<8 or mons%2 == 0 and mons>7:     
        print('---')
        for days in range(1,32,1):  #최대 31일까지 있기때문에 32로 설정
            if days<10:             #날짜가 10 이하일경우 앞에 0 추가
                Bdate = '0'+str(days)
            else:                   #그외 날짜는 그대로 저장
                Bdate = str(days)
            urlstack()
    elif mons == 2:                 #2월의 날짜는 혼자다름 
        print('---')
        for days in range(1,29,1):  
            if days<10:             #날짜가 10 이하일경우 앞에 0 추가
                Bdate = '0'+str(days)
            else:                   #그외 날짜는 그대로 저장
                Bdate = str(days)
            urlstack()
    elif mons%2 ==0 and mons<7 or mons%2 !=0 and mons>8:   
        print('---')
        for days in range(1,31,1):  #최대 30일까지 있기때문에 31로 설정
            if days<10:             #날짜가 10 이하일경우 앞에 0 추가
                Bdate = '0'+str(days)
            else:                   #그외 날짜는 그대로 저장
                Bdate = str(days)
            urlstack()
```

request로 페이지의 응답을 받아오는데
```
import requests
request = requests.get('http://www.naver.com/')
```
로 요청하여 print(request)를 하면 ```<Response [200]>``` 가나온다.<br>
그냥 200과는 다른방식

***
## 함수를 이용해 줄인 코드 1차

```
def zeroday():
    if days<10:             #날짜가 10 이하일경우 앞에 0 추가
        Bdate = '0'+str(days)
    else:                   #그외 날짜는 그대로 저장
        Bdate = str(days)
        
def urlstack():
    date = '2019' + str(Adate) + str(Bdate)
    url_date = url + '&date=' + date
    print(url_date)
    
def fordays():
    global days                   #전역변수를 지역 범위에서 사용하고 싶으면 global사용
    for days in range(1,days,1):  #최대 31일까지 있기때문에 32로 설정
            zeroday()
            urlstack()

for mons in range(1,13,1):      #최대 12얼까지 있기때문에 13으로 설정
    if mons<10:                 #월이 10 이하일 경우 앞에 0 추가
        Adate = '0' + str(mons)
    else:
        Adate = str(mons)
    
    if mons%2 != 0 and mons<8 or mons%2 == 0 and mons>7:     
        print('---')
        days = 32
        fordays()
    elif mons == 2:                 #2월의 날짜는 혼자다름 
        print('---')
        days = 28
        fordays()
    elif mons%2 ==0 and mons<7 or mons%2 !=0 and mons>8:   
        print('---')
        days = 31
        fordays()
```

***
## 함수를 이용해 줄인 코드 2차

In [13]:
def urlres(url):
    
    request = urllib.request.Request(url_date)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    if(rescode==200): #코드가 정상이면 200 즉, 200일때 실행
        response_body = response.read()
        xmlsoup = BeautifulSoup(response_body, 'html.parser')
        return xmlsoup
    else:
        return None

In [21]:
'''
def urlstack():
    if days<10:             #날짜가 10 이하일경우 앞에 0 추가
        Bdate = '0'+str(days)
    else:                   #그외 날짜는 그대로 저장
        Bdate = str(days)
    date = '2019' + str(Adate) + str(Bdate)
    url_date = url + '&date=' + date
    print(url_date)
    
    xmlsoup = urlres(url_date)
    pagePart = xmlsoup.find('div',{'class':'paging'})
    anchorNumber = pagePart.find_all({'a':'href'})
    pageNumber = len(anchorNumber) + 1
    print("페이지수",pageNumber)
    
    if pageNumber == 11:
        imsiUrl = url_date+'&page=11'
        xmlsoup = urlres(imsiUrl)
        pagePart = xmlsoup.find('div',{'class':'paging'})
        anchorNumber = pagePart.find_all({'a':'href'})
        pageNumber = len(anchorNumber) + 1
        print("페이지수",pageNumber)
        
    for page in range(1,pageNumber+1,1):
        newDayUrl = url_date + '&page=' + str(page)
        #print(newDayUrl)
'''
def fordays():
    global days                   #전역변수를 지역 범위에서 사용하고 싶으면 global사용
    for days in range(1,days,1):  #최대 31일까지 있기때문에 32로 설정
            #urlstack()
            

for mons in range(1,13,1):      #최대 12얼까지 있기때문에 13으로 설정
    if mons<10:                 #월이 10 이하일 경우 앞에 0 추가
        Adate = '0' + str(mons)
    else:
        Adate = str(mons)
    
    if mons%2 != 0 and mons<8 or mons%2 == 0 and mons>7:     
        days = 32 
    elif mons == 2:                 #2월의 날짜는 혼자다름 
        days = 28  
    elif mons%2 ==0 and mons<7 or mons%2 !=0 and mons>8:   
        days = 31  
    
    fordays()
    print('---')

IndentationError: expected an indented block (<ipython-input-21-4ac36c5a0769>, line 34)

In [22]:
def DateNum(mon):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-22-680cf750a530>, line 2)